In [1]:
from sklearn.datasets import fetch_20newsgroups
news = fetch_20newsgroups(subset='all',remove=('headers', 'footers', 'quotes'))

In [36]:
print type(news.data[0])
print type(news.target)
print type(news.target_names)

<type 'unicode'>
<type 'numpy.ndarray'>
<type 'list'>


In [3]:
print len(news.data)
print len(news.target)
print len(news.target_names)

18846
18846
20


In [4]:
print news.target_names[news.target[0]]
print news.data[0]
print type(news.data[0])

rec.sport.hockey


I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However, I am going to put an end
to non-PIttsburghers' relief with a bit of praise for the Pens. Man, they
are killing those Devils worse than I thought. Jagr just showed you why
he is much better than his regular season stats. He is also a lot
fo fun to watch in the playoffs. Bowman should let JAgr have a lot of
fun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final
regular season game.          PENS RULE!!!


<type 'unicode'>


In [5]:
from sklearn.model_selection import train_test_split

def train(model, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)
 
    model.fit(X_train, y_train)
    print "Accuracy: %s" % model.score(X_test, y_test)

In [6]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

### TF.IDF

### Pipeline 1
1. No Lemmatization
1. No Stop-Words
1. Unigrams 
1. No min/max document frequency limits

In [28]:
trial1 = Pipeline([
    ('vectorizer', TfidfVectorizer(input='content',encoding='utf-8', lowercase='true')),
    ('classifier', MultinomialNB(alpha=0.05)),
])
train(trial1,news.data,news.target)

Accuracy: 0.7554376657824934


### Pipeline 2
1. No Lemmatization
1. Yes Stop-Words
1. Unigrams 
1. No min/max document frequency limits

In [10]:
import nltk
nltk.download('stopwords')       # For Stopwords
stopwords = nltk.corpus.stopwords.words('English')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kapilkalra04\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [29]:
trial2 = Pipeline([
    ('vectorizer', TfidfVectorizer(input='content',encoding='utf-8', lowercase='true', stop_words=stopwords)),
    ('classifier', MultinomialNB(alpha=0.05)),
])
train(trial2,news.data,news.target)

Accuracy: 0.7615384615384615


### Pipeline 3
1. Yes Lemmatization
1. Yes Stop-Words
1. Unigrams 
1. No min/max document frequency limits

In [57]:
nltk.download('punkt')           # For Tokenizing
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag         # POS Tagger
from nltk.corpus import wordnet as wn

def penn_to_wn(tag):
    """
    Convert between the PennTreebank (pos_tag) tags to simple Wordnet tags
    """
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

wnl = WordNetLemmatizer()

def lemmatize(word, pos=wn.NOUN):
    return wnl.lemmatize(word,pos=pos)

def tokenize(text):
     return nltk.tokenize.word_tokenize(text)
    
def preprocess(text):
    preprocessedtext = []
    words = tokenize(text)
    tagged_text = pos_tag(words)
    for word, tag in tagged_text:
        word = word.lower()
        tag = penn_to_wn(tag)
        if tag in (wn.NOUN,wn.ADJ,wn.VERB,wn.ADV):
            root = lemmatize(word,tag)
            preprocessedtext.append(root)
    return preprocessedtext

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kapilkalra04\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [58]:
import string
stopwords.append(string.punctuation)

In [60]:
print preprocess(news.data[0])

[u'be', u'sure', u'bashers', u'pen', u'fan', u'be', u'pretty', u'confuse', u'lack', u'kind', u'post', u'recent', u'pen', u'massacre', u'devil', u'actually', u'be', u'bit', u'puzzled', u'too', u'bit', u'relieved', u'however', u'be', u'go', u'put', u'end', u'non-pittsburghers', u'relief', u'bit', u'praise', u'pen', u'man', u'be', u'kill', u'devil', u'bad', u'think', u'jagr', u'just', u'show', u'be', u'much', u'good', u'regular', u'season', u'stats', u'be', u'also', u'lot', u'fo', u'fun', u'watch', u'playoff', u'bowman', u'let', u'jagr', u'have', u'lot', u'fun', u'next', u'couple', u'game', u'pen', u'be', u'go', u'beat', u'pulp', u'jersey', u'anyway', u'be', u'very', u'disappointed', u'not', u'see', u'islander', u'lose', u'final', u'regular', u'season', u'game', u'pen', u'rule']


In [61]:
trial3 = Pipeline([
    ('vectorizer', TfidfVectorizer(input='content',encoding='utf-8', lowercase='true', 
                                   tokenizer=preprocess, stop_words=stopwords)),
    ('classifier', MultinomialNB(alpha=0.05)),
])
train(trial3,news.data,news.target)

Accuracy: 0.7514588859416446


### Pipeline 4
1. Yes Lemmatization
1. Yes Stop-Words
1. Unigrams and Bigrams 
1. No min/max document frequency limits

In [62]:
trial4 = Pipeline([
    ('vectorizer', TfidfVectorizer(input='content',encoding='utf-8', lowercase='true', 
                                   tokenizer=preprocess, stop_words=stopwords, ngram_range=(1,2))),
    ('classifier', MultinomialNB(alpha=0.05)),
])
train(trial4,news.data,news.target)

Accuracy: 0.7538461538461538
